__File: srg_operators_fig.ipynb__

__Author:   A. J. Tropiano (tropiano.4@osu.edu)__<br/>
__Date:     May 17, 2019__

This notebook creates several figures showing SRG-evolved operators (other than the potential itself). This is done by constructing the unitary transformation explicitly and applying to an operator. Each function creates and saves a figure to the __Figures/SRG_operator_evolution__ folder. Below is an outline of each function:
* deuteron_momentum_distribution - Plots a semi-log figure of deuteron momentum distributions comparing initial and SRG-evolved $\psi^2$. Note this depicts $\psi(s)^2$ not $\psi^{\dagger}(0) U^{\dagger}(s) U(s) \psi(0) = |\psi(0)|^2$.
* continuum_state_distribution - Analog of the deuteron_momentum_distribution function but for some state $\epsilon_k$ > 0 MeV.
* potential_contours - Plots contours of an SRG-evolving potential in $\lambda$.
* potential_diagonals_test - Checks the diagonal matrix elements of an SRG-evolving potential in $\lambda$.
* momentum_projection_contours - Plots contours of the momentum projection operator SRG-evolving in $\lambda$.
* momentum_projection_diagonals_test - Checks the diagonal matrix elements of the above figure.
* momentum_projection_integrand - Plots contours of the integrand of the inner product of the momentum projection operator with the deuteron wave function SRG-evolving in $\lambda$. <font color=red>Work in progress.</font>
* momentum_projection_integrand_test - Checks the diagonal matrix elements of the above figure. <font color=red>Work in progress.</font>
* r2_contours - Plots contours of the radius squared operator SRG-evolving in $\lambda$. <font color=red>Work in progress.</font>

__Revision history:<br/>__
* May 29, 2019 - Updated with changes to deuteron.py, operators.py, and observables.py.
* June 3, 2019 - Added a function for generating evolved potentials, potential_contours.
* August 26, 2019 - Finalized the SRG potential and momentum projetion operator evolution figures.
* September 23, 2019 - Removed redundant functions (e.g. potential_contours which is now in srg_potentials_fig.ipynb).

In [1]:
from os import chdir, getcwd
import matplotlib.colors as colors
from matplotlib.offsetbox import AnchoredText
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.linalg import expm
# Scripts made by A.T.
from Figures import figures_functions as ff
from Potentials.vsrg_macos import load_save_potentials as lp
import observables as ob
import operators as op
from SRG_codes.srg_unitary_transformation import SRG_unitary_transformation

__Plotting functions__

In [2]:
def momentum_projection_contours(q, kvnn, channel, kmax, kmid, ntot, generator, lambda_array):
    """
    Plots momentum projection operator contours [fm^6] with respect to momentum [fm^-1] SRG-evolving in lambda. Must
    specify the momentum value, q, to projection onto, and details of the SRG transformation, hence the arguments
    kvnn, channel, etc. This is a 1 x 4 sub-plots figure.
    
    Parameters
    ----------
    q : float
        Momentum value in units fm^-1.
    kvnn : int
        This number specifies the potential.
    channel : str
        The partial wave channel ('1S0', '3S1', etc.)
    kmax : float
        Maximum value in the momentum mesh.
    kmid : float
        Mid-point value in the momentum mesh.
    ntot : int
        Number of momentum points in mesh.
    generator : str
        SRG generator 'Wegner', 'T', or 'Block-diag'.
    lambda_array : 1-D ndarray
        Lambda evolution values in units fm^-1.

    Saves
    -----
    file_name = momentum_projection_contours_ with an extension based off the given arguments. For example,
                momentum_projection_contours_q3,00_kvnn10_3S1_Block-diag.pdf.
    
    """
    
    
    # --- Set-up --- #
    
    # Load initial Hamiltonian, momentum, and weights
    H_initial = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
    k_array, k_weights = lp.load_momentum(kvnn, channel, kmax, kmid, ntot)
    # The arrays below are used later to present a mesh-independent result
    factor_array = k_array * np.sqrt(k_weights) * np.sqrt(2/np.pi)
    row, col = np.meshgrid(factor_array, factor_array)
    
    # Limits of axes on contours (units are fm^-1)
    if q >= 4.0:
        axes_max = 5.0
    else:
        axes_max = 4.0
        
    # Specifications of x and y axes
    # Step-size in labeling tick marks
    axes_stepsize = 1.0
    # x and y axes ticks
    axes_ticks = np.arange(0.0, axes_max + axes_stepsize, axes_stepsize)
        
    # Labels
    axes_label = 'k [fm' + r'$^{-1}$' + ']'
    # Label the block-diagonal Lambda
    if generator == 'Block-diag':
        lambda_label = r'$\Lambda=%.1f$' + ' fm' + r'$^{-1}$'
    # For band-diagonal generators, label lambda
    else:
        lambda_label = r'$\lambda=%.1f$' + ' fm' + r'$^{-1}$'
    generator_label = ff.generator_label_conversion(generator)
    colorbar_label = '[fm' + r'$^6$' + ']'
    
    # Fontsize for labels and tick marks
    axes_label_size = 18
    lambda_label_size = 17
    generator_label_size = 17
    colorbar_label_size = 20
    axes_tick_size = 18
    colorbar_tick_size = 18
        
    # Limits of colorbar (units are fm^6) and location of labels (dependent on specified q value)
    if q < axes_max/2:
        mx = 0.1
        mn = -0.1
        generator_label_location = 'center right'
        lambda_label_location = 'lower right'
    elif q >= 4.0:
        mx = 1e-6
        mn = -1e-6
        generator_label_location = 'upper left'
        lambda_label_location = 'lower left'
    else:
        mx = 0.01
        mn = -0.01
        generator_label_location = 'upper left'
        lambda_label_location = 'lower left'
        
    # Color scheme for contour plots
    color_style = 'jet'
    
    # Size of figure
    row_number = 1
    col_number = len(lambda_array)
    figure_size = (4*col_number, 3.5*row_number) # (width, height) - extra width for colorbar
    
    # File name of figure
    # This reads 'momentum_projection_contours_q_value_kvnn_number_channel_generator'
    file_name = 'momentum_projection_contours_q%.2f_kvnn%d_%s_%s' % (q, kvnn, channel, generator)

    # Current working directory
    cwd = getcwd()
        

    # --- Load data and plot contours --- #
    
    # Initialize figure
    plt.close('all')
    f, axs = plt.subplots(row_number, col_number, sharex=True, sharey=True, figsize=figure_size)
    
    # Loop over lambda values keeping track of sub-plot number i
    i = 0
    for lamb in lambda_array:
        
        # Load unitary transformation
        # SRG calls function which builds U(s) out of un-evolved and evolved eigenvectors
        if generator == 'Block-diag':
            H_evolved = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, 'srg', generator, lambda_array[-1],
                                            lambda_bd=lamb)
        else:
            H_evolved = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, 'srg', generator, lamb)
        U_matrix = SRG_unitary_transformation(H_initial, H_evolved)
        
        # Evolved momentum projection operator
        operator = op.momentum_projection_operator(q, k_array, k_weights, channel, U_matrix)
        # Take only the upper sub-block if coupled-channel 
        if lp.coupled_channel(channel):
            operator = operator[:ntot, :ntot]
        # Divide by k_i * k_j * sqrt( w_i * w_j ) for mesh-independent result
        operator = operator / row / col
        
        # Interpolate the operator through 0 to axes_max for smoother looking figure (the extension _int means 
        # interpolated)
        k_array_int, operator_int = ff.interpolate_matrix(k_array, operator, axes_max)
    
        # Add sub-plot to figure (for last sub-plot, must specify colorbar c)
        if i != ( len(lambda_array) - 1):
            axs[i].pcolormesh(k_array_int, k_array_int, operator_int, cmap=color_style, vmin=mn, vmax=mx,
                              rasterized=True)
        else:
            c = axs[i].pcolormesh(k_array_int, k_array_int, operator_int, cmap=color_style, vmin=mn, vmax=mx,
                                  rasterized=True)
    
        # Specify axes tick marks
        axs[i].xaxis.set_ticks(axes_ticks)
        axs[i].xaxis.set_ticklabels(axes_ticks)
        
        # Specify axes limits
        axs[i].set_xlim( (0, axes_max) )
        axs[i].set_ylim( (0, axes_max) )
        
        # Position of x-axis label and tick marks
        axs[i].xaxis.set_label_position('top')
        axs[i].xaxis.tick_top()
        axs[i].tick_params(labeltop=True, labelsize=axes_tick_size)
        
        # Prevent overlapping x-axis tick marks unless it's the last sub-plot
        if i != ( len(lambda_array) - 1 ):
            xticks = axs[i].xaxis.get_major_ticks()
            xticks[-1].set_visible(False)
            
        # Set axes labels
        axs[i].set_xlabel(axes_label, fontsize=axes_label_size)
        # Only specify y axis tick marks and set label for 1st sub-plot and add generator label as anchored text
        # A
        if i == 0:
            axs[i].yaxis.set_ticks(axes_ticks)
            axs[i].yaxis.set_ticklabels(axes_ticks)
            axs[i].set_ylabel(axes_label, fontsize=axes_label_size)
            generator_anchored_text = AnchoredText(generator_label, prop=dict(size=generator_label_size),
                                                   loc=generator_label_location)
            axs[i].add_artist(generator_anchored_text)
            
        # Add lambda label as anchored text
        lambda_anchored_text = AnchoredText(lambda_label % lambda_array[i], prop=dict(size=lambda_label_size),
                                            loc=lambda_label_location)
        axs[i].add_artist(lambda_anchored_text)
            
        i += 1
    

    # --- Set figure specifications and save --- #

    # Invert y-axis
    plt.gca().invert_yaxis()
    # Amount of white space in-between sub-plots
    f.subplots_adjust(hspace=0.0, wspace=0.0)
    # Adjust for colorbar space
    f.subplots_adjust(right=0.8)
    cbar_ax = f.add_axes( (0.85, 0.15, 0.05, 0.7) )
    # Add colorbar and set tick size
    cbar = f.colorbar(c, cax=cbar_ax)
    cbar.ax.tick_params(labelsize=colorbar_tick_size)
    # Set colorbar label
    cbar.ax.set_title(colorbar_label, fontsize=colorbar_label_size)

    # Replace '.' with ',' in file name since LaTeX doesn't like periods
    file_name = ff.replace_periods_with_commas(file_name)
    
    # Save figure
    chdir('Figures/SRG_operators')
    f.savefig(file_name+'.pdf', bbox_inches='tight')
    chdir(cwd)

In [3]:
def momentum_projection_diagonals_test(q, kvnn, channel, kmax_1, kmid_1, ntot_1, kmax_2, kmid_2, ntot_2, generator,
                                       lambda_bd=0.00):  
    """
    Plots a 1 x 4 sub-plots figure comparing two momentum projection operators using different momentum meshes 
    evolving from lambda = 6.0 to 1.5 fm^-1 showing the diagonal matrix elements. Must specify details of the 
    SRG-evolved operator, hence the arguments kvnn, channel, etc.
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    channel : str
        The partial wave channel ('1S0', '3S1', etc.)
    kmax_1 : float
        Maximum value in the first momentum mesh.
    kmid_1 : float
        Mid-point value in the first momentum mesh.
    ntot_1 : int
        Number of momentum points in first mesh.
    kmax_2 : float
        Maximum value in the second momentum mesh.
    kmid_2 : float
        Mid-point value in the second momentum mesh.
    ntot_2 : int
        Number of momentum points in second mesh.
    generator : str
        SRG generator 'Wegner', 'T', or 'Block-diag'.
    lambda_bd : float, optional
        Lambda value for block-diagonal decoupling (e.g. 2.00 fm^-1).
    
    """
    
    # Current working directory
    cwd = getcwd()
    
    
    # Store evolved potentials
    
    # Load initial Hamiltonians, momentum and weight arrays
    H_initial_1 = lp.load_hamiltonian(kvnn, channel, kmax_1, kmid_1, ntot_1)
    H_initial_2 = lp.load_hamiltonian(kvnn, channel, kmax_2, kmid_2, ntot_2)
    k_array_1, k_weights_1 = lp.load_momentum(kvnn, channel, kmax_1, kmid_1, ntot_1)
    k_array_2, k_weights_2 = lp.load_momentum(kvnn, channel, kmax_2, kmid_2, ntot_2)
    # The arrays below are used later to present a mesh-independent result
    factor_array_1 = k_array_1 * np.sqrt(k_weights_1) * np.sqrt(2/np.pi)
    row_1, col_1 = np.meshgrid(factor_array_1, factor_array_1)
    factor_array_2 = k_array_2 * np.sqrt(k_weights_2) * np.sqrt(2/np.pi)
    row_2, col_2 = np.meshgrid(factor_array_2, factor_array_2)
    
    # Loop over lambda values in lambdas and save evolved potentials to dictionary where the keys are lambda values
    lambdas = [6.0, 3.0, 2.0, 1.5]
    
    # Initialize dictionaries
    d = {}
    d[1] = {}
    d[2] = {}
    
    for lamb in lambdas:
        
        # Load unitary transformations
        # SRG calls function which builds U(s) out of un-evolved and evolved eigenvectors
        H_evolved_1 = lp.load_hamiltonian(kvnn, channel, kmax_1, kmid_1, ntot_1, 'srg', generator, lamb, lambda_bd)
        U_matrix_1 = SRG_unitary_transformation(H_initial_1, H_evolved_1)
        H_evolved_2 = lp.load_hamiltonian(kvnn, channel, kmax_2, kmid_2, ntot_2, 'srg', generator, lamb, lambda_bd)
        U_matrix_2 = SRG_unitary_transformation(H_initial_2, H_evolved_2)
        
        # Evolved momentum projection operators in 3S1 channel divided by k_i * k_j * sqrt( w_i * w_j ) for 
        # mesh-independent result
        operator_1 = op.momentum_projection_operator(q, k_array_1, k_weights_1, U_matrix_1)[:ntot_1, :ntot_1] \
                     / row_1 / col_1
        operator_2 = op.momentum_projection_operator(q, k_array_2, k_weights_2, U_matrix_2)[:ntot_2, :ntot_2] \
                     / row_2 / col_2

        
        # Save diagonal elements of the operator to dictionary
        d[1][lamb] = np.diag(operator_1)
        d[2][lamb] = np.diag(operator_2)
        

    # Plot specifications
    
    # Labels
    x_label = r'$k \/ \/ [fm^{-1}]$'
    y_label = r'$a_q^{\dagger} a_q$'
    if generator == 'Wegner':
        generator_label = r'$G=H_{D}$'
    elif generator == 'T':
        generator_label = r'$G=T_{rel}$'
    lamb_label = r'$\lambda=%.1f \/ fm^{-1}$'
    
    # Fontsize for labels and tick marks
    x_label_size = 20
    y_label_size = 24
    generator_label_size = 22
    lamb_label_size = 18
    legend_label_size = 16
    axes_tick_size = 14
     
    # Location of labels
    generator_label_location = 3
    legend_label_location = 6
    lamb_label_location = 2
    
    # Limits of axes
    if q >= 4.0:
        x_max = 5.0
    else:
        x_max = 4.0 # Units of fm^-1
    if q >= 4.0:
        y_max = 1e-5
        y_min = -1e-6
    else:
        y_max = 0.01
        y_min = -0.0025
        
    # Step-size in labeling tick marks
    #x_stepsize = 1
    #y_stepsize = 0.25*y_max
    
    # x and y axes ticks
    #x_ticks = np.arange(0.0, x_max + x_stepsize, x_stepsize)
    #y_ticks = np.arange(y_min, y_max + y_stepsize, y_stepsize)
    
    # Size of figure
    row_number = 1
    column_number = 4
    figure_size = (3.5*column_number, 3.5*row_number) # (width, height)
    
    
    # Plot the 1 x 4 sub-plots figure     
    plt.close('all')
    
    f, (ax1, ax2, ax3, ax4) = plt.subplots(row_number, column_number, sharex=True, sharey=True, figsize=figure_size)
    
    # Sub-plot 1: lambda = 6.0 fm^-1
    ax1.plot(k_array_1, d[1][ lambdas[0] ], color='xkcd:red', label='Mesh 1')
    ax1.plot(k_array_2, d[2][ lambdas[0] ], color='xkcd:black', linestyle='dotted', label='Mesh 2')
    # Set limits of axes
    ax1.set_xlim( (0, x_max) )
    ax1.set_ylim( (y_min, y_max) )
    # Set axes labels
    ax1.set_xlabel(x_label, fontsize=x_label_size)
    ax1.set_ylabel(y_label, fontsize=y_label_size)
    # Specify y axis tick marks
    #ax1.yaxis.set_ticks(y_ticks)
    #ax1.yaxis.set_ticklabels(y_ticks)
    # Set fontsize of tick marks
    ax1.tick_params(labelsize=axes_tick_size)
    # Prevent overlapping axes tick marks
    xticks_1 = ax1.xaxis.get_major_ticks()
    xticks_1[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_1 = AnchoredText(lamb_label%lambdas[0], prop=dict(size=lamb_label_size), loc=lamb_label_location,
                                   frameon=False)
    ax1.add_artist(anchored_text_1)
    # Set legend
    ax1.legend(loc=legend_label_location, frameon=False, fontsize=legend_label_size)
    
    # Sub-plot 2: lambda = 3.0 fm^-1
    ax2.plot(k_array_1, d[1][ lambdas[1] ], color='xkcd:red')
    ax2.plot(k_array_2, d[2][ lambdas[1] ], color='xkcd:black', linestyle='dotted')
    # Set x label
    ax2.set_xlabel(x_label, fontsize=x_label_size)
    ax2.set_xlim( (0, x_max) )
    ax2.set_ylim( (y_min, y_max) )
    # Set fontsize of tick marks
    ax2.tick_params(labelsize=axes_tick_size)
    # Prevent overlapping axes tick marks
    xticks_2 = ax2.xaxis.get_major_ticks()
    xticks_2[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_2 = AnchoredText(lamb_label%lambdas[1], prop=dict(size=lamb_label_size), loc=lamb_label_location,
                                   frameon=False)
    ax2.add_artist(anchored_text_2)
    
    # Sub-plot 3: lambda = 2.0 fm^-1
    ax3.plot(k_array_1, d[1][ lambdas[2] ], color='xkcd:red')
    ax3.plot(k_array_2, d[2][ lambdas[2] ], color='xkcd:black', linestyle='dotted')
    # Set x label
    ax3.set_xlabel(x_label, fontsize=x_label_size)
    # Set limits of axes
    ax3.set_xlim( (0, x_max) )
    ax3.set_ylim( (y_min, y_max) )
    # Set fontsize of tick marks
    ax3.tick_params(labelsize=axes_tick_size)
    # Prevent overlapping axes tick marks
    xticks_3 = ax3.xaxis.get_major_ticks()
    xticks_3[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_3 = AnchoredText(lamb_label%lambdas[2], prop=dict(size=lamb_label_size), loc=lamb_label_location,
                                   frameon=False)
    ax3.add_artist(anchored_text_3)
    
    # Sub-plot 4: lambda = 1.5 fm^-1
    ax4.plot(k_array_1, d[1][ lambdas[3] ], color='xkcd:red')
    ax4.plot(k_array_2, d[2][ lambdas[3] ], color='xkcd:black', linestyle='dotted')
    # Set x label
    ax4.set_xlabel(x_label, fontsize=x_label_size)
    # Set limits of axes
    ax4.set_xlim( (0, x_max) )
    ax4.set_ylim( (y_min, y_max) )
    # Set fontsize of tick marks
    ax4.tick_params(labelsize=axes_tick_size)
    # Add lambda label onto plot as anchored text
    anchored_text_4 = AnchoredText(lamb_label%lambdas[3], prop=dict(size=lamb_label_size), loc=lamb_label_location,
                                   frameon=False)
    ax4.add_artist(anchored_text_4)
    anchored_text_5 = AnchoredText(generator_label, prop=dict(size=generator_label_size),
                                   loc=generator_label_location, frameon=False)
    ax4.add_artist(anchored_text_5)

    # Amount of white space in-between sub-plots
    f.subplots_adjust(hspace=0.0, wspace=0.0)

    
    # Testing
    plt.show()

In [4]:
def momentum_projection_slices(line, q, channel, lambda_array, *transformations):
    """
    Plots the momentum projection operators [fm^6] with respect to momentum [fm^-1] SRG-evolving in lambda isolating
    either the diagonal or far off-diagonal matrix elements. Must specify the momentum value, q, to projection onto, 
    the partial wave channel, lambda values, and each SRG transformation of interest. Here, we compare the Wegner
    generator at the different lambda values to the block-diagonal generator at Lambda_BD = lambda (Wegner) and
    fixing lambda (block diagonal) to the last value of lambda in lambda_array.
    
    Parameters
    ----------
    line : str
        The slice of V(k, k') 'diag' or 'off-diag'.
    q : float
        Momentum value in units fm^-1.
    channel : str
        The partial wave channel ('1S0', '3S1', etc.)
    lambda_array : 1-D ndarray
        Lambda evolution values in units fm^-1. These values will also be used as Lambda_BD values for the block-
        diagonal generator.
    *transformations : tuple
        Specifications for SRG transformations in the format (kvnn, kmax, kmid, ntot) where kvnn (int) is the number
        specifying the potential, kmax (float) is the maximum value in the momentum mesh, kmid (float) is the
        mid-point value in the mesh, and ntot (int) is the number of points in the mesh. For example,
        (222, 10.0, 2.0, 120) corresponds to the default Gezerlis et al. N2LO potential.

    Saves
    -----
    file_name = momentum_projection_ with an extension based off the given arguments. For example,
                momentum_projection_diags_q3,00_3S1_kvnns_10_106_222_lamb1,0.pdf.
    
    """
    
    
    # Check that at least one transformation was provided
    if not transformations: 
        print('At least one SRG transformation must be specified.')
        return None

    
    # --- Set-up --- #
    
    # Limits of x and y axes and location of legend (dependent on q value)
    if q >= 4.0:
        xlim = [0.0, 5.0]
        ylim = [-1e-6, 1e-5]
        legend_label_location = 'upper left'
    elif q <= 2.0:
        xlim = [0.0, 4.0]
        ylim = [-0.025, 0.1]
        legend_label_location = 'upper right'
    else:
        xlim = [0.0, 4.0]
        ylim = [-0.0025, 0.01]
        legend_label_location = 'upper left'
        
    # Labels
    x_label = 'k [fm' + r'$^{-1}$' + ']'
    if line == 'diag':
        y_label = r'$a^{\dagger}_q a_q(k,k)$' + ' [fm' + r'$^6$' + ']'
    elif line == 'off-diag':
        y_label = r'$a^{\dagger}_q a_q(k,0)$' + ' [fm' + r'$^6$' + ']'
    # Label for lambda and Lambda_BD
    lambda_label = r'$\lambda$' + ', ' + r'$\Lambda=%.1f$' + ' fm' + r'$^{-1}$'
    
    # Fontsize for labels and tick marks
    x_label_size = 18
    y_label_size = 20
    legend_label_size = 14
    lambda_label_size = 17
    
    # Location of labels
    lambda_label_location = 'lower left'
    
    # Size of figure
    row_number = 1
    col_number = len(lambda_array)
    figure_size = (4*col_number, 4*row_number) # (width, height)
    
    # Initialize file name of figure
    # This reads 'momentum_projection_line_q_value_channel_kvnns'
    file_name = 'momentum_projection_%s_q%.2f_%s_kvnns' % (line, q, channel)
    # These lists will keep track of kvnn numbers and generators to be added to the file name
    kvnn_list = []

    # Current working directory
    cwd = getcwd()


    # --- Load data and plot lines --- #
    
    # Initialize figure
    plt.close('all')
    f, axs = plt.subplots(row_number, col_number, sharex=True, sharey=True, figsize=figure_size)
    
    # Loop over SRG-evolved potentials starting with the generator
    for generator in ['Wegner', 'Block-diag']:
        # Next potential specifications
        for transformation in transformations:
        
            # Set kvnn, kmax, kmid, ntot, and generator
            kvnn = transformation[0]
            kmax = transformation[1]
            kmid = transformation[2]
            ntot = transformation[3]
        
            # Keep track of kvnn numbers for file name at the end
            if kvnn not in kvnn_list:
                kvnn_list.append(kvnn)
            
            # Load initial Hamiltonian and momentum
            H_initial = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
            k_array, k_weights = lp.load_momentum(kvnn, channel, kmax, kmid, ntot)
            # The arrays below are used to present a mesh-independent result
            factor_array = k_array * np.sqrt(k_weights) * np.sqrt(2/np.pi)
            row, col = np.meshgrid(factor_array, factor_array)
        
            # Curve labels and styles
            
            # Curve color depends on potential
            if kvnn in [10, 900]:
                curve_color = 'xkcd:black'
            elif kvnn in [105, 106, 107, 110, 111, 112, 901]:
                curve_color = 'xkcd:red'
            elif kvnn in [222, 224, 902]:
                curve_color = 'xkcd:blue'
                
            # Curve style depends on band- or block-diagonal generator
            # Put potential label on 1st sub-plot
            if generator == 'Block-diag':
                curve_style = 'dashdot'
                potential_label = '' # No need to redundantly label the potentials
            else:
                curve_style = 'solid'
                potential_label = ff.kvnn_label_conversion(kvnn) # Legend label on 1st sub-plot
                
            # Put generator label on last sub-plot
            if kvnn == kvnn_list[0]:
                generator_label = ff.generator_label_conversion(generator)
            else:
                generator_label = '' # No need to redundantly label generators
        
            # Loop over lambda
            i = 0 # Sub-plot number
            for lamb in lambda_array:

                # Load unitary transformations
                # SRG calls function which builds U(s) out of un-evolved and evolved eigenvectors
                if generator == 'Block-diag':
                    H_evolved = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, 'srg', generator,
                                                    lambda_array[-1], lambda_bd=lamb)
                else:
                    H_evolved = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, 'srg', generator, lamb)
                    
                # Calculate the unitary transformation
                U_matrix = SRG_unitary_transformation(H_initial, H_evolved)
        
                # Evolved momentum projection operator
                operator_matrix = op.momentum_projection_operator(q, k_array, k_weights, channel, U_matrix)
                # Take only the upper sub-block if coupled-channel 
                if lp.coupled_channel(channel):
                    operator_matrix = operator_matrix[:ntot, :ntot]
                # Divide by k_i * k_j * sqrt( w_i * w_j ) for mesh-independent result
                operator_matrix = operator_matrix / row / col
                    
                # Take a slice of the operator to plot
                if line == 'diag':
                    operator_vector = np.diag(operator_matrix)
                elif line == 'off-diag':
                    operator_vector = operator_matrix[:, 0]
 
                # Add sub-plot to figure
                if i == 0: # 1st sub-plot - label potential
                    axs[i].plot(k_array, operator_vector, color=curve_color, linestyle=curve_style,
                                label=potential_label)
                elif i == ( len(lambda_array) - 1 ): # Last sub-plot - label generator
                    axs[i].plot(k_array, operator_vector, color=curve_color, linestyle=curve_style,
                                label=generator_label)
                else: # Middle sub-plots - no labels
                    axs[i].plot(k_array, operator_vector, color=curve_color, linestyle=curve_style)
                
                i += 1
        
        
    # --- Set figure specifications and save --- #
    
    # Loop over sub-plots
    for j in range( len(lambda_array) ):
        
        # Specify axes limits
        axs[j].set_xlim(xlim)
        axs[j].set_ylim(ylim)
        
        # Prevent overlapping x-axis tick marks unless it's the last sub-plot
        if j != ( len(lambda_array) - 1 ):
            xticks = axs[j].xaxis.get_major_ticks()
            xticks[-1].set_visible(False)
            
        # Set axes labels
        axs[j].set_xlabel(x_label, fontsize=x_label_size)
        
        # Only specify y label and potential labels as legend for 1st sub-plot
        if j == 0:
            # Add y label
            axs[j].set_ylabel(y_label, fontsize=y_label_size)
            # Add legend for potentials
            axs[j].legend(loc=legend_label_location, frameon=False, fontsize=legend_label_size)
            
        # Specify generator labels as legend for last sub-plot
        if j == ( len(lambda_array) - 1 ):
            # Add legend for potentials
            axs[j].legend(loc=legend_label_location, frameon=False, fontsize=legend_label_size)
            
        # Add lambda's label as anchored text
        lambda_anchored_text = AnchoredText(lambda_label % lambda_array[j], prop=dict(size=lambda_label_size),
                                            loc=lambda_label_location, frameon=False)
        # Add lambda
        axs[j].add_artist(lambda_anchored_text)
    
    # Amount of white space in-between sub-plots
    f.subplots_adjust(hspace=0.0, wspace=0.0)
    
    # Name of the file
    for kvnn in kvnn_list:
        file_name += '_%d' % kvnn
    # Add last value of lambda to file name
    file_name += '_lamb%.1f' % lambda_array[-1]
    # Replace '.' with ',' in file name since LaTeX doesn't like periods
    file_name = ff.replace_periods_with_commas(file_name)
    
    # Save figure
    chdir('Figures/SRG_operators')
    f.savefig(file_name+'.pdf', bbox_inches='tight')
    chdir(cwd)

In [ ]:
# Work in progress
def momentum_projection_integrand(q, kvnn, channel, kmax, kmid, ntot, generator, lambda_bd=0.00):
    """
    Plots a 1 x 4 sub-plots figure of the integrand of an SRG-evolved momentum projection operator sandwiched with
    a consistently evolved wave function evolving from lambda = 6.0 to 1.5 fm^-1 as contour plots. Must specify 
    details of the SRG-evolved potential, hence the arguments kvnn, channel, etc.
    
    Parameters
    ----------
    q : float
        Momentum value in units fm^-1.
    kvnn : int
        This number specifies the potential.
    channel : str
        The partial wave channel ('1S0', '3S1', etc.)
    kmax : float
        Maximum value in the momentum mesh.
    kmid : float
        Mid-point value in the momentum mesh.
    ntot : int
        Number of momentum points in mesh.
    generator : str
        SRG generator 'Wegner', 'T', or 'Block-diag'.
    lambda_bd : float, optional
        Lambda value for block-diagonal decoupling (e.g. 2.00 fm^-1).

    Saves
    -----
    file_name = momentum_projection_integrand_ with an extension based off the given arguments.
    file_type = .pdf
    
    """
    
    # Current working directory
    cwd = getcwd()
    
    # Limits of axes on contours (units are fm^-1)
    axes_max = 4.0
    
    # Store evolved integrands
    
    # Load initial Hamiltonian, momentum, and weights
    H_initial = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
    k_array, k_weights = lp.load_momentum(kvnn, channel, kmax, kmid, ntot)
    #k_array_double = np.concatenate( (k_array, k_array) )
    #k_weights_double = np.concatenate( (k_weights, k_weights) )
    # The arrays below are used later to present a mesh-independent result 
    factor_array = k_array * np.sqrt(k_weights) * np.sqrt(2/np.pi)
    factor_array = np.concatenate((factor_array, factor_array))
    row, col = np.meshgrid(factor_array, factor_array)
    
    # Loop over lambda values in lambdas and save evolved operators to dictionary where the keys are lambda values
    lambdas = [6.0, 3.0, 2.0, 1.5]
    
    # Initialize dictionary
    d = {}
    
    for lamb in lambdas:
        
        # Load unitary transformation
        # SRG calls function which builds U(s) out of un-evolved and evolved eigenvectors
        H_evolved = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, 'srg', generator, lamb, lambda_bd)
        U_matrix = SRG_unitary_transformation(H_initial, H_evolved)
        
        # Evolved momentum projection operator in 3S1 channel divided by k_i * k_j * sqrt( w_i * w_j ) for 
        # mesh-independent result
        operator = op.momentum_projection_operator(q, k_array, k_weights, U_matrix)
        operator = op.momentum_projection_operator(q, k_array, k_weights, U_matrix) / row / col
        
        # Load evolved wave functions and divide out momentum and weights
        #psi = ob.wave_function(H_initial, eps=-2.22, U=U_matrix)
        psi = ob.wave_function(H_initial, eps=-2.22, U=U_matrix) / factor_array * np.sqrt(2/np.pi)
        # Mesh-grids of psi
        psi_row, psi_col = np.meshgrid(psi, psi)
        
        # Integrand of < psi | a^dagger_q a_q | psi >
        integrand = psi_row * operator * psi_col
        
        # Re-size to S-S block
        integrand = integrand[:ntot, :ntot]
        # For log plot take absolute value
        #integrand = abs(integrand)
        
        # Interpolate the integrand through 0 to axes_max for smoother looking figure (the extension _int means 
        # interpolated)
        k_array_int, integrand_int = ff.interpolate_matrix(k_array, integrand, axes_max)
        #k_array_int = k_array
        #integrand_int = integrand
        
        # Save integrand to dictionary
        d[lamb] = integrand_int
    
    
    # Plot specifications
    
    # Labels
    k_label = r'$k \/ \/ [fm^{-1}]$'
    #q_label = r'$q=%.2f \/ fm^{-1}$'%q # This label can be written explicitly in the caption of the figure
    if generator == 'Block-diag':
        generator_label = r'$G=H_{BD}$'+'\n'+r'$\Lambda=%.2f \/ fm^{-1}$'%lambda_bd
    elif generator == 'Wegner':
        generator_label = r'$G=H_{D}$'
    elif generator == 'T':
        generator_label = r'$G=T_{rel}$'
    lamb_label = r'$\lambda=%.1f \/ fm^{-1}$'
    colorbar_label = r'$fm^3$'
    
    # Fontsize for labels and tick marks
    axes_label_size = 18
    #q_label_size = 18
    generator_label_size = 17
    lambda_label_size = 16
    colorbar_label_size = 18
    axes_tick_size = 18
    colorbar_tick_size = 18
        
    # Location of labels (dependent on specified q value)
    if q < axes_max/2:
        #q_location = 7
        generator_location = 'center right'
        lamb_location = 'lower right'
    else:
        #q_location = 2
        generator_location = 'upper right'
        lamb_location = 'lower center'
        
    # Color scheme for contour plots
    color_style = 'Blues'
    #color_style = 'seismic'
    #color_style = 'jet'
    
    # Limits of color bar in fm and colorbar step-size (dependent on specified q value)
    if q < axes_max/2:
        mx = 1.0
        mn = -1.0
    else:
        #mx = 1e-3
        #mx = 1e-4
        #mx = 5e-6
        mx = 1e-2
        #mx = 1e-1
        #mn = -1e-3
        #mn = -1e-4
        #mn = -5e-6
        #mn = -1e-2
        mn = 1e-8
        
    # Step-size in colorbar ticks
    color_bar_stepsize = 0.5*mx
        
    # Colorbar ticks and labels (the value of the ticks are floats and the labels are strings)
    colorbar_ticks = np.arange(mn, mx + color_bar_stepsize, color_bar_stepsize)
    colorbar_ticks_list = []
    for tick in colorbar_ticks:
        tick_str = '%.1e'%tick
        #tick_str = '%.2f'%tick
        colorbar_ticks_list.append(tick_str)
        
    # Step-size in labeling tick marks
    axes_stepsize = 1.0
    
    # x and y axes ticks
    axes_ticks = np.arange(0.0, axes_max + axes_stepsize, axes_stepsize)
    
    
    # Plot the 1 x 4 sub-plots figure     
    plt.close('all')
    
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(16,3.5))
    
    # Sub-plot 1: lambda = 6.0 fm^-1
    ax1.pcolormesh(k_array_int, k_array_int, d[ lambdas[0] ], cmap=color_style, 
                   norm=colors.LogNorm(vmin=mn, vmax=mx), rasterized=True)
    #ax1.pcolormesh(k_array_int, k_array_int, d[ lambdas[0] ], cmap=color_style, vmin=mn, vmax=mx, rasterized=True)
    # Set limits of axes
    ax1.set_xlim( (0, axes_max) )
    ax1.set_ylim( (0, axes_max) )
    # Set x and y labels
    ax1.set_xlabel(k_label, fontsize=axes_label_size)
    ax1.set_ylabel(k_label, fontsize=axes_label_size)
    # Specify axes tick marks
    ax1.xaxis.set_ticks(axes_ticks)
    ax1.xaxis.set_ticklabels(axes_ticks)
    ax1.yaxis.set_ticks(axes_ticks)
    ax1.yaxis.set_ticklabels(axes_ticks)
    # Position of x-axis labels and tick marks
    ax1.xaxis.set_label_position('top')
    ax1.xaxis.tick_top()
    ax1.tick_params(labeltop=True, labelsize=axes_tick_size)
    # Prevent overlapping x-axis tick marks
    xticks_1 = ax1.xaxis.get_major_ticks()
    xticks_1[-1].set_visible(False)
    # Add generator and lambda labels onto plot as anchored texts
    anchored_text_1 = AnchoredText(lamb_label%lambdas[0], prop=dict(size=lambda_label_size), loc=lamb_location)
    ax1.add_artist(anchored_text_1)
    
    # Sub-plot 2: lambda = 3.0 fm^-1
    ax2.pcolormesh(k_array_int, k_array_int, d[ lambdas[1] ], cmap=color_style,
                   norm=colors.LogNorm(vmin=mn, vmax=mx), rasterized=True)
    #ax2.pcolormesh(k_array_int, k_array_int, d[ lambdas[1] ], cmap=color_style, vmin=mn, vmax=mx, rasterized=True)
    # Set limits of axes
    ax2.set_xlim( (0, axes_max) )
    ax2.set_ylim( (0, axes_max) )
    # Set x label
    ax2.set_xlabel(k_label, fontsize=axes_label_size)
    # Specify axes tick marks
    ax2.xaxis.set_ticks(axes_ticks)
    ax2.xaxis.set_ticklabels(axes_ticks)
    # Position of x-axis labels and tick marks
    ax2.xaxis.set_label_position('top')
    ax2.xaxis.tick_top()
    ax2.tick_params(labeltop=True, labelsize=axes_tick_size)
    # Prevent overlapping x-axis tick marks
    xticks_2 = ax2.xaxis.get_major_ticks()
    xticks_2[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_2 = AnchoredText(lamb_label%lambdas[1], prop=dict(size=lambda_label_size), loc=lamb_location)
    ax2.add_artist(anchored_text_2)
    
    # Sub-plot 3: lambda = 2.0 fm^-1
    ax3.pcolormesh(k_array_int, k_array_int, d[ lambdas[2] ], cmap=color_style,
                   norm=colors.LogNorm(vmin=mn, vmax=mx), rasterized=True)
    #ax3.pcolormesh(k_array_int, k_array_int, d[ lambdas[2] ], cmap=color_style, vmin=mn, vmax=mx, rasterized=True)
    # Set limits of axes
    ax3.set_xlim( (0, axes_max) )
    ax3.set_ylim( (0, axes_max) )
    # Set x label
    ax3.set_xlabel(k_label, fontsize=axes_label_size)
    # Specify axes tick marks
    ax3.xaxis.set_ticks(axes_ticks)
    ax3.xaxis.set_ticklabels(axes_ticks)
    # Position of x-axis labels and tick marks
    ax3.xaxis.set_label_position('top')
    ax3.xaxis.tick_top()
    ax3.tick_params(labeltop=True, labelsize=axes_tick_size)
    # Prevent overlapping x-axis tick marks
    xticks_3 = ax3.xaxis.get_major_ticks()
    xticks_3[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_3 = AnchoredText(lamb_label%lambdas[2], prop=dict(size=lambda_label_size), loc=lamb_location)
    ax3.add_artist(anchored_text_3)

    # Sub-plot 4: lambda = 1.5 fm^-1
    c = ax4.pcolormesh(k_array_int, k_array_int, d[ lambdas[3] ], cmap=color_style,
                       norm=colors.LogNorm(vmin=mn, vmax=mx), rasterized=True)
    #c = ax4.pcolormesh(k_array_int, k_array_int, d[ lambdas[3] ], cmap=color_style, vmin=mn, vmax=mx, 
                       #rasterized=True)
    # Set limits of axes
    ax4.set_xlim( (0, axes_max) )
    ax4.set_ylim( (0, axes_max) )
    # Set x label
    ax4.set_xlabel(k_label, fontsize=axes_label_size)
    # Specify axes tick marks
    ax4.xaxis.set_ticks(axes_ticks)
    ax4.xaxis.set_ticklabels(axes_ticks)
    # Position of x-axis labels and tick marks
    ax4.xaxis.set_label_position('top')
    ax4.xaxis.tick_top()
    ax4.tick_params(labeltop=True, labelsize=axes_tick_size)
    # Add lambda label onto plot as anchored text
    anchored_text_4 = AnchoredText(lamb_label%lambdas[3], prop=dict(size=lambda_label_size), loc=lamb_location)
    ax4.add_artist(anchored_text_4)
    anchored_text_5 = AnchoredText(generator_label, prop=dict(size=generator_label_size), loc=generator_location)
    ax1.add_artist(anchored_text_5)

    # Invert y-axis
    plt.gca().invert_yaxis()
    # Amount of white space in-between sub-plots
    f.subplots_adjust(hspace=0.0, wspace=0.0)
    # Adjust for colorbar space
    f.subplots_adjust(right=0.8)
    cbar_ax = f.add_axes([0.85, 0.15, 0.05, 0.7])
    # Add colorbar and set ticks
    cbar = f.colorbar(c, cax=cbar_ax)
    #cbar.ax.set_title(colorbar_label, fontsize=colorbar_label_size)
    #cbar.set_ticks(colorbar_ticks)
    #cbar.set_ticklabels(colorbar_ticks_list)
    cbar.ax.tick_params(labelsize=colorbar_tick_size)

    
    # Save figure as a .pdf
    
    # Name of the file
    if generator == 'Block-diag':
        name = 'momentum_projection_integrand_q%.2f_kvnn%d_%s_%s%.2f'%(q, kvnn, channel, generator, lambda_bd)
    else:
        name = 'momentum_projection_integrand_q%.2f_kvnn%d_%s_%s'%(q, kvnn, channel, generator)
        
    # Replace '.' with ',' in file name since LaTeX doesn't like periods
    file_name = ff.replace_periods_with_commas(name)
        
    chdir('Figures/SRG_operator_evolution')
    f.savefig(file_name+'.pdf', bbox_inches='tight')
    chdir(cwd)
    
    # For testing purposes
    plt.show()

In [ ]:
# Work in progress
def momentum_projection_integrand_test(q, kvnn, channel, kmax_1, kmid_1, ntot_1, kmax_2, kmid_2, ntot_2, generator,
                                       lambda_bd=0.00):  
    """
    Plots a 1 x 4 sub-plots figure comparing two integrands corresponding to an SRG-evolved momentum projection 
    operator sandwiched with a consistently evolved wave function using different momentum meshes evolving from 
    lambda = 6.0 to 1.5 fm^-1 showing the diagonal matrix elements. Must specify details of the SRG-evolved 
    operator, hence the arguments kvnn, channel, etc.
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    channel : str
        The partial wave channel ('1S0', '3S1', etc.)
    kmax_1 : float
        Maximum value in the first momentum mesh.
    kmid_1 : float
        Mid-point value in the first momentum mesh.
    ntot_1 : int
        Number of momentum points in first mesh.
    kmax_2 : float
        Maximum value in the second momentum mesh.
    kmid_2 : float
        Mid-point value in the second momentum mesh.
    ntot_2 : int
        Number of momentum points in second mesh.
    generator : str
        SRG generator 'Wegner', 'T', or 'Block-diag'.
    lambda_bd : float, optional
        Lambda value for block-diagonal decoupling (e.g. 2.00 fm^-1).
    
    """
    
    # Current working directory
    cwd = getcwd()
    
    
    # Store evolved potentials
    
    # Load initial Hamiltonians, momentum and weight arrays
    H_initial_1 = lp.load_hamiltonian(kvnn, channel, kmax_1, kmid_1, ntot_1)
    H_initial_2 = lp.load_hamiltonian(kvnn, channel, kmax_2, kmid_2, ntot_2)
    k_array_1, k_weights_1 = lp.load_momentum(kvnn, channel, kmax_1, kmid_1, ntot_1)
    k_array_2, k_weights_2 = lp.load_momentum(kvnn, channel, kmax_2, kmid_2, ntot_2)
    # The arrays below are used later to present a mesh-independent result
    factor_array_1 = k_array_1 * np.sqrt(k_weights_1) * np.sqrt(2/np.pi)
    factor_array_1 = np.concatenate((factor_array_1, factor_array_1))
    row_1, col_1 = np.meshgrid(factor_array_1, factor_array_1)
    factor_array_2 = k_array_2 * np.sqrt(k_weights_2) * np.sqrt(2/np.pi)
    factor_array_2 = np.concatenate((factor_array_2, factor_array_2))
    row_2, col_2 = np.meshgrid(factor_array_2, factor_array_2)
    
    # Loop over lambda values in lambdas and save evolved potentials to dictionary where the keys are lambda values
    lambdas = [6.0, 3.0, 2.0, 1.5]
    
    # Initialize dictionaries
    d = {}
    d[1] = {}
    d[2] = {}
    
    for lamb in lambdas:
        
        # Load unitary transformations
        # SRG calls function which builds U(s) out of un-evolved and evolved eigenvectors
        H_evolved_1 = lp.load_hamiltonian(kvnn, channel, kmax_1, kmid_1, ntot_1, 'srg', generator, lamb, lambda_bd)
        U_matrix_1 = SRG_unitary_transformation(H_initial_1, H_evolved_1)
        H_evolved_2 = lp.load_hamiltonian(kvnn, channel, kmax_2, kmid_2, ntot_2, 'srg', generator, lamb, lambda_bd)
        U_matrix_2 = SRG_unitary_transformation(H_initial_2, H_evolved_2)
        
        # Evolved momentum projection operators in 3S1 channel divided by k_i * k_j * sqrt( w_i * w_j ) for 
        # mesh-independent result
        #operator_1 = op.momentum_projection_operator(q, k_array_1, k_weights_1, U_matrix_1)
        operator_1 = op.momentum_projection_operator(q, k_array_1, k_weights_1, U_matrix_1) / row_1 / col_1
        #operator_2 = op.momentum_projection_operator(q, k_array_2, k_weights_2, U_matrix_2)
        operator_2 = op.momentum_projection_operator(q, k_array_2, k_weights_2, U_matrix_2) / row_2 / col_2
        
        # Load evolved wave functions and divide out momentum and weights
        #psi_1 = ob.wave_function(H_initial_1, eps=-2.22, U=U_matrix_1)
        psi_1 = ob.wave_function(H_initial_1, eps=-2.22, U=U_matrix_1) / factor_array_1 * np.sqrt(2/np.pi)
        # Mesh-grids of psi
        psi_row_1, psi_col_1 = np.meshgrid(psi_1, psi_1)
        #psi_2 = ob.wave_function(H_initial_2, eps=-2.22, U=U_matrix_2)
        psi_2 = ob.wave_function(H_initial_2, eps=-2.22, U=U_matrix_2) / factor_array_2 * np.sqrt(2/np.pi)
        # Mesh-grids of psi
        psi_row_2, psi_col_2 = np.meshgrid(psi_2, psi_2)
        # Integrand of < psi | a^dagger_q a_q | psi >
        integrand_1 = psi_row_1 * operator_1 * psi_col_1
        integrand_2 = psi_row_2 * operator_2 * psi_col_2

        # Save diagonal elements of the integrand to dictionary
        d[1][lamb] = np.diag(integrand_1[:ntot_1, :ntot_1])
        d[2][lamb] = np.diag(integrand_2[:ntot_2, :ntot_2])
        

    # Plot specifications
    
    # Labels
    x_label = r'$k \/ \/ [fm^{-1}]$'
    y_label = r'$\psi(k_i) a_q^{\dagger} a_q (k_i, k_j) \psi(k_j)$'
    if generator == 'Wegner':
        generator_label = r'$G=H_{D}$'
    elif generator == 'T':
        generator_label = r'$G=T_{rel}$'
    lamb_label = r'$\lambda=%.1f \/ fm^{-1}$'
    
    # Fontsize for labels and tick marks
    x_label_size = 20
    y_label_size = 20
    generator_label_size = 22
    lamb_label_size = 18
    legend_label_size = 16
    axes_tick_size = 14
     
    # Location of labels
    generator_label_location = 4
    legend_label_location = 7
    lamb_label_location = 1
    
    # Limits of axes
    x_max = 4.0 # Units of fm^-1
    #y_max = 5e-6
    y_max = 1e0
    #y_min = -2.5e-6
    #y_min = -1e-3
    y_min = 1e-7
        
    # Step-size in labeling tick marks
    #x_stepsize = 1
    #y_stepsize = 0.25*y_max
    
    # x and y axes ticks
    #x_ticks = np.arange(0.0, x_max + x_stepsize, x_stepsize)
    #y_ticks = np.arange(y_min, y_max + y_stepsize, y_stepsize)
    
    # Size of figure
    row_number = 1
    column_number = 4
    figure_size = (3.5*column_number, 3.5*row_number) # (width, height)
    
    
    # Plot the 1 x 4 sub-plots figure     
    plt.close('all')
    
    f, (ax1, ax2, ax3, ax4) = plt.subplots(row_number, column_number, sharex=True, sharey=True, figsize=figure_size)
    
    # Sub-plot 1: lambda = 6.0 fm^-1
    #ax1.plot(k_array_1, d[1][ lambdas[0] ], color='xkcd:red', label='Mesh 1')
    ax1.semilogy(k_array_1, d[1][ lambdas[0] ], color='xkcd:red')
    #ax1.plot(k_array_2, d[2][ lambdas[0] ], color='xkcd:black', linestyle='dotted', label='Mesh 2')
    ax1.semilogy(k_array_2, d[2][ lambdas[0] ], color='xkcd:black', linestyle='dotted')
    # Set limits of axes
    ax1.set_xlim( (0, x_max) )
    ax1.set_ylim( (y_min, y_max) )
    # Set axes labels
    ax1.set_xlabel(x_label, fontsize=x_label_size)
    ax1.set_ylabel(y_label, fontsize=y_label_size)
    # Specify y axis tick marks
    #ax1.yaxis.set_ticks(y_ticks)
    #ax1.yaxis.set_ticklabels(y_ticks)
    # Set fontsize of tick marks
    ax1.tick_params(labelsize=axes_tick_size)
    # Prevent overlapping axes tick marks
    xticks_1 = ax1.xaxis.get_major_ticks()
    xticks_1[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_1 = AnchoredText(lamb_label%lambdas[0], prop=dict(size=lamb_label_size), loc=lamb_label_location,
                                   frameon=False)
    ax1.add_artist(anchored_text_1)
    
    # Sub-plot 2: lambda = 3.0 fm^-1
    #ax2.plot(k_array_1, d[1][ lambdas[1] ], color='xkcd:red')
    ax2.semilogy(k_array_1, d[1][ lambdas[1] ], color='xkcd:red')
    #ax2.plot(k_array_2, d[2][ lambdas[1] ], color='xkcd:black', linestyle='dotted')
    ax2.semilogy(k_array_2, d[2][ lambdas[1] ], color='xkcd:black', linestyle='dotted')
    # Set x label
    ax2.set_xlabel(x_label, fontsize=x_label_size)
    ax2.set_xlim( (0, x_max) )
    ax2.set_ylim( (y_min, y_max) )
    # Set fontsize of tick marks
    ax2.tick_params(labelsize=axes_tick_size)
    # Prevent overlapping axes tick marks
    xticks_2 = ax2.xaxis.get_major_ticks()
    xticks_2[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_2 = AnchoredText(lamb_label%lambdas[1], prop=dict(size=lamb_label_size), loc=lamb_label_location,
                                   frameon=False)
    ax2.add_artist(anchored_text_2)
    
    # Sub-plot 3: lambda = 2.0 fm^-1
    #ax3.plot(k_array_1, d[1][ lambdas[2] ], color='xkcd:red')
    ax3.semilogy(k_array_1, d[1][ lambdas[2] ], color='xkcd:red', label='Mesh 1')
    #ax3.plot(k_array_2, d[2][ lambdas[2] ], color='xkcd:black', linestyle='dotted')
    ax3.semilogy(k_array_2, d[2][ lambdas[2] ], color='xkcd:black', linestyle='dotted', label='Mesh 2')
    # Set x label
    ax3.set_xlabel(x_label, fontsize=x_label_size)
    # Set limits of axes
    ax3.set_xlim( (0, x_max) )
    ax3.set_ylim( (y_min, y_max) )
    # Set fontsize of tick marks
    ax3.tick_params(labelsize=axes_tick_size)
    # Prevent overlapping axes tick marks
    xticks_3 = ax3.xaxis.get_major_ticks()
    xticks_3[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_3 = AnchoredText(lamb_label%lambdas[2], prop=dict(size=lamb_label_size), loc=lamb_label_location,
                                   frameon=False)
    ax3.add_artist(anchored_text_3)
    # Set legend
    ax3.legend(loc=legend_label_location, frameon=False, fontsize=legend_label_size)
    
    # Sub-plot 4: lambda = 1.5 fm^-1
    #ax4.plot(k_array_1, d[1][ lambdas[3] ], color='xkcd:red')
    ax4.semilogy(k_array_1, d[1][ lambdas[3] ], color='xkcd:red')
    #ax4.plot(k_array_2, d[2][ lambdas[3] ], color='xkcd:black', linestyle='dotted')
    ax4.semilogy(k_array_2, d[2][ lambdas[3] ], color='xkcd:black', linestyle='dotted')
    # Set x label
    ax4.set_xlabel(x_label, fontsize=x_label_size)
    # Set limits of axes
    ax4.set_xlim( (0, x_max) )
    ax4.set_ylim( (y_min, y_max) )
    # Set fontsize of tick marks
    ax4.tick_params(labelsize=axes_tick_size)
    # Add lambda label onto plot as anchored text
    anchored_text_4 = AnchoredText(lamb_label%lambdas[3], prop=dict(size=lamb_label_size), loc=lamb_label_location,
                                   frameon=False)
    ax4.add_artist(anchored_text_4)
    anchored_text_5 = AnchoredText(generator_label, prop=dict(size=generator_label_size),
                                   loc=generator_label_location, frameon=False)
    ax4.add_artist(anchored_text_5)

    # Amount of white space in-between sub-plots
    f.subplots_adjust(hspace=0.0, wspace=0.0)

    
    # Testing
    plt.show()

In [ ]:
# Work in progress
def r2_contours(kvnn, channel, kmax, kmid, ntot, generator, lambda_bd=0.00):
    """
    Plots a 1 x 4 sub-plots figure of the r^2 operator in momentum-space evolving from lambda = 6.0 to 1.5 fm^-1 as
    contour plots. Must specify details of the SRG-evolved potential, hence the arguments kvnn, channel, etc.
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    channel : str
        The partial wave channel ('1S0', '3S1', etc.)
    kmax : float
        Maximum value in the momentum mesh.
    kmid : float
        Mid-point value in the momentum mesh.
    ntot : int
        Number of momentum points in mesh.
    generator : str
        SRG generator 'Wegner', 'T', or 'Block-diag'.
    lambda_bd : float, optional
        Lambda value for block-diagonal decoupling (e.g. 2.00 fm^-1).

    Saves
    -----
    file_name = r2_contours_ with an extension based off the given arguments.
    file_type = .pdf
    
    """
    # Current working directory
    cwd = getcwd()
    
    # Limits of axes on contours (units are fm^-1)
    axes_max = 0.4
    
    # Store evolved operators
    
    # Load initial Hamiltonian, momentum, and weights
    H_initial = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
    k_array, k_weights = lp.load_momentum(kvnn, channel, kmax, kmid, ntot)
    # The arrays below are used later to present a mesh-independent result 
    row, col = np.meshgrid(k_array*np.sqrt(k_weights), k_array*np.sqrt(k_weights))
    
    # Specify r_array
    r_min = 0.005
    r_max = 30.2
    dr = 0.005
    r_array = np.arange(r_min, r_max + dr, dr)
    
    # Loop over lambda values in lambdas and save evolved operators to dictionary where the keys are lambda values
    lambdas = [6.0, 3.0, 2.0, 1.5]
    
    # Initialize dictionary
    d = {}
    
    for lamb in lambdas:
        
        # Load unitary transformation
        # SRG calls function which builds U(s) out of un-evolved and evolved eigenvectors
        H_evolved = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, 'srg', generator, lamb, lambda_bd)
        U_matrix = SRG_unitary_transformation(H_initial, H_evolved)
        
        # Evolved r^2 operator in 3S1 channel divided by k_i * k_j * sqrt( w_i * w_j ) for 
        # mesh-independent result
        operator = op.r2_operator(k_array, k_weights, r_array, dr, U_matrix)[:ntot, :ntot]
        #operator = op.r2_operator(k_array, k_weights, r_array, dr, U_matrix)[:ntot, :ntot] / row / col
        
        # Interpolate the operator through 0 to axes_max for smoother looking figure (the extension _int means 
        # interpolated)
        k_array_int, operator_int = ff.interpolate_matrix(k_array, operator, axes_max)
        
        # Save operator to dictionary
        d[lamb] = operator_int
        
        
    # Plot specifications
    
    # Labels
    k_label = r'$k \/ \/ [fm^{-1}]$'
    if generator == 'Block-diag':
        generator_label = r'$G=H_{BD}$'+'\n'+r'$\Lambda=%.2f \/ fm^{-1}$'%lambda_bd
    elif generator == 'Wegner':
        generator_label = r'$G=H_{D}$'
    elif generator == 'T':
        generator_label = r'$G=T_{rel}$'
    lamb_label = r'$\lambda=%.1f \/ fm^{-1}$'
    
    # Fontsize for labels and tick marks
    axes_label_size = 18
    generator_label_size = 17
    lambda_label_size = 17
    axes_tick_size = 18
    colorbar_tick_size = 18
        
    # Location of labels
    generator_location = 1
    lamb_location = 3
        
    # Color scheme for contour plots
    color_style = 'jet'
    
    # Limits of color bar in fm and colorbar step-size
    mx = 1e6
    mn = -1e6
        
    # Step-size in colorbar ticks
    color_bar_stepsize = 0.5*mx
        
    # Colorbar ticks and labels (the value of the ticks are floats and the labels are strings)
    colorbar_ticks = np.arange(mn, mx + color_bar_stepsize, color_bar_stepsize)
    colorbar_ticks_list = []
    for tick in colorbar_ticks:
        tick_str = '%.1e'%tick
        colorbar_ticks_list.append(tick_str)
        
    # Step-size in labeling tick marks
    axes_stepsize = 0.1
    
    # x and y axes ticks
    axes_ticks = np.arange(0.0, axes_max + axes_stepsize, axes_stepsize)
    axes_ticks_list = []
    for tick in axes_ticks:
        axes_ticks_list.append('%.1f'%tick)
    
    
    # Plot the 1 x 4 sub-plots figure     
    plt.close('all')
    
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(16,3.5))
    
    # Sub-plot 1: lambda = 6.0 fm^-1
    ax1.pcolormesh(k_array_int, k_array_int, d[ lambdas[0] ], cmap=color_style, vmin=mn, vmax=mx, rasterized=True)
    # Set limits of axes
    ax1.set_xlim( (0, axes_max) )
    ax1.set_ylim( (0, axes_max) )
    # Set x and y labels
    ax1.set_xlabel(k_label, fontsize=axes_label_size)
    ax1.set_ylabel(k_label, fontsize=axes_label_size)
    # Specify axes tick marks
    ax1.xaxis.set_ticks(axes_ticks)
    ax1.xaxis.set_ticklabels(axes_ticks_list)
    ax1.yaxis.set_ticks(axes_ticks)
    ax1.yaxis.set_ticklabels(axes_ticks_list)
    # Position of x-axis labels and tick marks
    ax1.xaxis.set_label_position('top')
    ax1.xaxis.tick_top()
    ax1.tick_params(labeltop=True, labelsize=axes_tick_size)
    # Prevent overlapping x-axis tick marks
    xticks_1 = ax1.xaxis.get_major_ticks()
    xticks_1[-1].set_visible(False)
    # Add generator and lambda labels onto plot as anchored texts
    anchored_text_1 = AnchoredText(generator_label, prop=dict(size=generator_label_size), loc=generator_location)
    ax1.add_artist(anchored_text_1)
    anchored_text_2 = AnchoredText(lamb_label%lambdas[0], prop=dict(size=lambda_label_size), loc=lamb_location)
    ax1.add_artist(anchored_text_2)
    
    # Sub-plot 2: lambda = 3.0 fm^-1
    ax2.pcolormesh(k_array_int, k_array_int, d[ lambdas[1] ], cmap=color_style, vmin=mn, vmax=mx, rasterized=True)
    # Set limits of axes
    ax2.set_xlim( (0, axes_max) )
    ax2.set_ylim( (0, axes_max) )
    # Set x label
    ax2.set_xlabel(k_label, fontsize=axes_label_size)
    # Specify axes tick marks
    ax2.xaxis.set_ticks(axes_ticks)
    ax2.xaxis.set_ticklabels(axes_ticks_list)
    # Position of x-axis labels and tick marks
    ax2.xaxis.set_label_position('top')
    ax2.xaxis.tick_top()
    ax2.tick_params(labeltop=True, labelsize=axes_tick_size)
    # Prevent overlapping x-axis tick marks
    xticks_2 = ax2.xaxis.get_major_ticks()
    xticks_2[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_3 = AnchoredText(lamb_label%lambdas[1], prop=dict(size=lambda_label_size), loc=lamb_location)
    ax2.add_artist(anchored_text_3)
    
    # Sub-plot 3: lambda = 2.0 fm^-1
    ax3.pcolormesh(k_array_int, k_array_int, d[ lambdas[2] ], cmap=color_style, vmin=mn, vmax=mx, rasterized=True)
    # Set limits of axes
    ax3.set_xlim( (0, axes_max) )
    ax3.set_ylim( (0, axes_max) )
    # Set x label
    ax3.set_xlabel(k_label, fontsize=axes_label_size)
    # Specify axes tick marks
    ax3.xaxis.set_ticks(axes_ticks)
    ax3.xaxis.set_ticklabels(axes_ticks_list)
    # Position of x-axis labels and tick marks
    ax3.xaxis.set_label_position('top')
    ax3.xaxis.tick_top()
    ax3.tick_params(labeltop=True, labelsize=axes_tick_size)
    # Prevent overlapping x-axis tick marks
    xticks_3 = ax3.xaxis.get_major_ticks()
    xticks_3[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_4 = AnchoredText(lamb_label%lambdas[2], prop=dict(size=lambda_label_size), loc=lamb_location)
    ax3.add_artist(anchored_text_4)

    # Sub-plot 4: lambda = 1.5 fm^-1
    c = ax4.pcolormesh(k_array_int, k_array_int, d[ lambdas[3] ], cmap=color_style, vmin=mn, vmax=mx, 
                       rasterized=True)
    # Set limits of axes
    ax4.set_xlim( (0, axes_max) )
    ax4.set_ylim( (0, axes_max) )
    # Set x label
    ax4.set_xlabel(k_label, fontsize=axes_label_size)
    # Specify axes tick marks
    ax4.xaxis.set_ticks(axes_ticks)
    ax4.xaxis.set_ticklabels(axes_ticks_list)
    # Position of x-axis labels and tick marks
    ax4.xaxis.set_label_position('top')
    ax4.xaxis.tick_top()
    ax4.tick_params(labeltop=True, labelsize=axes_tick_size)
    # Add lambda label onto plot as anchored text
    anchored_text_5 = AnchoredText(lamb_label%lambdas[3], prop=dict(size=lambda_label_size), loc=lamb_location)
    ax4.add_artist(anchored_text_5)

    # Invert y-axis
    plt.gca().invert_yaxis()
    # Amount of white space in-between sub-plots
    f.subplots_adjust(hspace=0.0, wspace=0.0)
    # Adjust for colorbar space
    f.subplots_adjust(right=0.8)
    cbar_ax = f.add_axes([0.85, 0.15, 0.05, 0.7])
    # Add colorbar and set ticks
    cbar = f.colorbar(c, cax=cbar_ax)
    cbar.set_ticks(colorbar_ticks)
    cbar.set_ticklabels(colorbar_ticks_list)
    cbar.ax.tick_params(labelsize=colorbar_tick_size)

    
    # Save figure as a .pdf
    
    # Name of the file
    if generator == 'Block-diag':
        name = 'r2_contours_kvnn%d_%s_%s%.2f'%(kvnn, channel, generator, lambda_bd)
    else:
        name = 'r2_contours_kvnn%d_%s_%s'%(kvnn, channel, generator)
        
    # Replace '.' with ',' in file name since LaTeX doesn't like periods
    file_name = ff.replace_periods_with_commas(name)
        
    chdir('Figures/SRG_operator_evolution')
    f.savefig(file_name+'.pdf', bbox_inches='tight')
    chdir(cwd)

__Run functions in the cell below:__

In [ ]:
# Fixed variables


kmax = 10.0
kmid = 2.0
ntot = 120

# Potentials formatted as (kvnn, kmax, kmid, ntot) tuples
# EM N3LO non-local potential at cutoff of 500 MeV
em_n3lo = (10, kmax, kmid, ntot)
# RKE N3LO semi-local potential at cutoff of 450 MeV
rke_n3lo = (106, kmax, kmid, ntot)
# Gezerlis et al N2LO local at cutoff 1.0 fm
gez_n2lo = (222, kmax, kmid, ntot)

# SRG lambda's array
lambda_array = np.array( [6.0, 3.0, 2.0, 1.5] ) # For contours

In [ ]:
# Plot SRG-evolved momentum projection operators


# Project to q = 3 fm^-1
q = 3.0

# AV18 with Wegner
#momentum_projection_contours(q, 10, '3S1', 30.0, 4.0, ntot, 'Wegner', lambda_array)

# EM N3LO non-local potential at cutoff of 500 MeV
# Wegner
momentum_projection_contours(q, 10, '3S1', kmax, kmid, ntot, 'Wegner', lambda_array)
# Block-diagonal
momentum_projection_contours(q, 10, '3S1', kmax, kmid, ntot, 'Block-diag', lambda_array)
# Wegner but for q = 0.3 fm^-1
momentum_projection_contours(0.3, 10, '3S1', kmax, kmid, ntot, 'Wegner', lambda_array)
# Wegner but in the 1S0 channel
momentum_projection_contours(q, 10, '1S0', kmax, kmid, ntot, 'Wegner', lambda_array)
# Wegner but in the 1P1 channel
momentum_projection_contours(q, 10, '1P1', kmax, kmid, ntot, 'Wegner', lambda_array)

# RKE N3LO semi-local potential at cutoff of 450 MeV
# Wegner
momentum_projection_contours(q, 106, '3S1', kmax, kmid, ntot, 'Wegner', lambda_array)
# Block-diagonal
momentum_projection_contours(q, 106, '3S1', kmax, kmid, ntot, 'Block-diag', lambda_array)
# Wegner but for q = 0.3 fm^-1
momentum_projection_contours(0.3, 106, '3S1', kmax, kmid, ntot, 'Wegner', lambda_array)
# Wegner but in the 1S0 channel
momentum_projection_contours(q, 106, '1S0', kmax, kmid, ntot, 'Wegner', lambda_array)
# Wegner but in the 1P1 channel
momentum_projection_contours(q, 106, '1P1', kmax, kmid, ntot, 'Wegner', lambda_array)

# Gezerlis et al N2LO local at cutoff 1.0 fm
# Wegner
momentum_projection_contours(q, 222, '3S1', kmax, kmid, ntot, 'Wegner', lambda_array)
# Block-diagonal
momentum_projection_contours(q, 222, '3S1', kmax, kmid, ntot, 'Block-diag', lambda_array)
# Wegner but for q = 0.3 fm^-1
momentum_projection_contours(0.3, 222, '3S1', kmax, kmid, ntot, 'Wegner', lambda_array)
# Wegner but in the 1S0 channel
momentum_projection_contours(q, 222, '1S0', kmax, kmid, ntot, 'Wegner', lambda_array)
# Wegner but in the 1P1 channel
momentum_projection_contours(q, 222, '1P1', kmax, kmid, ntot, 'Wegner', lambda_array)

In [ ]:
# Plot diagonal and off-diagonal matrix elements of SRG-evolved momentum projection operators


# Diagonal matrix elements for SRG-evolved momentum projection operators at q = 3 fm^-1
# 1S0
momentum_projection_slices('diag', q, '1S0', lambda_array, em_n3lo, rke_n3lo, gez_n2lo)
# 3S1
momentum_projection_slices('diag', q, '3S1', lambda_array, em_n3lo, rke_n3lo, gez_n2lo)
# 1P1
momentum_projection_slices('diag', q, '1P1', lambda_array, em_n3lo, rke_n3lo, gez_n2lo)
# 3S1 but q = 0.3 fm^-1
momentum_projection_slices('diag', 0.3, '3S1', lambda_array, em_n3lo, rke_n3lo, gez_n2lo)

# Far off-diagonal matrix elements for SRG-evolved momentum projection operators at q = 3 fm^-1
# 1S0
momentum_projection_slices('off-diag', q, '1S0', lambda_array, em_n3lo, rke_n3lo, gez_n2lo)
# 3S1
momentum_projection_slices('off-diag', q, '3S1', lambda_array, em_n3lo, rke_n3lo, gez_n2lo)
# 1P1 off-diagonals are mostly zero

In [ ]:
# Plot momentum projection integrands


# Project to q = 3 fm^-1
#q = 0.3
q = 3.0

# EM N3LO non-local potential
# Wegner
#print('Default mesh: kmax = 30 fm^-1 and kmid = 4 fm^-1')
#momentum_projection_integrand(q, 10, channel, 30.0, 4.0, ntot, 'Wegner')
# Wegner again but at k_max = 8 and k_mid = 2 fm^-1
#print('New mesh: kmax = 8 fm^-1 and kmid = 2 fm^-1')
#momentum_projection_integrand(q, 10, channel, 8.0, 2.0, ntot, 'Wegner')
# Wegner but different mesh
#print('kmax = 8 fm^-1, kmid = 2 fm^-1, ntot = 120')
#momentum_projection_integrand(q, 10, channel, 8.0, 2.0, ntot, 'Wegner')
#print('kmax = 8 fm^-1, kmid = 2 fm^-1, ntot = 140')
#momentum_projection_integrand(q, 10, channel, 8.0, 2.0, 140, 'Wegner')


# TESTING
#print('Mesh 1: kmax = 8 fm^-1 and kmid = 2 fm^-1 with ntot=120')
#print('Mesh 2: kmax = 8 fm^-1 and kmid = 2 fm^-1 with ntot=140')
#momentum_projection_integrand_test(q, 10, '3S1', 8.0, 2.0, 120, 8.0, 2.0, 140, 'Wegner')
#print('Mesh 1: kmax = 8 fm^-1 and kmid = 2 fm^-1 with ntot=120')
#print('Mesh 2: kmax = 30 fm^-1 and kmid = 4 fm^-1 with ntot=120')
#momentum_projection_integrand_test(q, 10, '3S1', 8.0, 2.0, 120, 30.0, 4.0, 120, 'Wegner')

In [ ]:
# Plot SRG-evolved  r^2 operators


# EM N3LO non-local potential
# Wegner
#r2_contours(10, channel, 30.0, 4.0, ntot, 'Wegner')

In [ ]:
# OLD STUFF IS BELOW

__Momentum distributions__

* Plotting functions for comparing initial and evolved momentum distributions.

In [ ]:
def deuteron_momentum_distribution(kvnn, kmax, kmid, ntot, generator, lamb, lambda_bd=0.00):
    """
    Plots a semi-log figure of the deuteron momentum distribution |psi_d(k)|^2 [fm^3] as a function of momentum k 
    [fm^-1] with psi_d(0) from the bare Hamiltonian and psi_d(lambda) from an evolved Hamiltonian at a given 
    value of lambda. Must specify details of the SRG-evolved potential, hence the arguments kvnn, channel, etc.
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    kmax : float
        Maximum value in the momentum mesh.
    kmid : float
        Mid-point value in the momentum mesh.
    ntot : int
        Number of momentum points in mesh.
    generator : str
        SRG generator 'Wegner', 'T', or 'Block-diag'.
    lamb : float
        Evolution parameter lambda in units fm^-1.
    lambda_bd : float, optional
        Lambda value for block-diagonal decoupling (e.g. 2.00 fm^-1).

    Saves
    -----
    file_name = deuteron_momentum_distribution_ with an extension based off the given arguments.
    file_type = .pdf
    
    """
    
    # Current working directory
    cwd = getcwd()
    
    
    # Load momentum distributions
    
    # Channel is 3S1 for deuteron
    channel = '3S1'
    # Deuteron energy in MeV
    eps = -2.22
    
    # Load initial Hamiltonian, momentum, and weights
    H_initial = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
    k_array, k_weights = lp.load_momentum(kvnn, channel, kmax, kmid, ntot) 
    
    # Load unitary transformation
    # SRG calls function which builds U(s) out of un-evolved and evolved eigenvectors
    H_evolved = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, 'srg', generator, lamb, lambda_bd)
    U_matrix = SRG_unitary_transformation(H_initial, H_evolved)
        
    # Compute initial and evolved wave functions
    psi_initial = ob.wave_function(H_initial, eps)
    u_initial = psi_initial[:ntot] # 3S1 component
    w_initial = psi_initial[ntot:] # 3D1 component
    
    psi_evolved = ob.wave_function(H_initial, eps, U=U_matrix)
    u_evolved = psi_evolved[:ntot] # 3S1 component
    w_evolved = psi_evolved[ntot:] # 3D1 component
    
    # Initial and evolved momentum distribution (divide by momenta and weights for mesh-independent result)
    phi_squared_initial = ( u_initial**2 + w_initial**2 ) / ( k_array**2 * k_weights )
    phi_squared_evolved = ( u_evolved**2 + w_evolved**2 ) / ( k_array**2 * k_weights )
    
    
    # Plot specifications

    # Limits of x and y axes
    xlim = [0.0, 4.0]
    ylim = [1e-5, 1e3]
    
    # Labels
    k_label = r'$k \/ [fm^{-1}]$'
    phi_squared_label = r'$\phi_d^2 \/ \/ [fm^3]$'
    lamb_label = r'$\lambda=%.1f \/ \/ fm^{-1}$'
    if generator == 'Block-diag':
        generator_label = r'$G=H_{BD}$'+'\n'+r'$\Lambda=%.2f \/ fm^{-1}$'%lambda_bd
    elif generator == 'Wegner':
        generator_label = r'$G=H_{D}$'
    elif generator == 'T':
        generator_label = r'$G=T_{rel}$'
    
    # Fontsize for labels
    legend_label_size = 16
    k_label_size = 18
    phi_squared_label_size = 20
    generator_label_size = 20
    
    # Location of labels
    legend_label_location = 'upper right'
    generator_label_location = 'lower left'
    
    
    # Plot the semi-log figure
    plt.close('all')
    
    f, ax = plt.subplots()
    
    ax.semilogy(k_array, phi_squared_evolved, color='xkcd:red', label=lamb_label%lamb)
    ax.semilogy(k_array, phi_squared_initial, color='xkcd:black', linestyle='dotted',
                label=r'$\lambda=\infty \/ \/ fm^{-1}$')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.legend(loc=legend_label_location, frameon=False, fontsize=legend_label_size)
    ax.set_xlabel(k_label, fontsize=k_label_size)
    ax.set_ylabel(phi_squared_label, fontsize=phi_squared_label_size)
    anchored_text = AnchoredText(generator_label, prop=dict(size=generator_label_size),
                                 loc=generator_label_location, frameon=False)
    ax.add_artist(anchored_text)
    
    
    # Save figure
    
    # Name of the file
    if generator == 'Block-diag':
        name = 'deuteron_momentum_distribution_kvnn%d_%s%.2f_lamb%.1f'%(kvnn, generator, lambda_bd, lamb)
    else:
        name = 'deuteron_momentum_distribution_kvnn%d_%s_lamb%.1f'%(kvnn, generator, lamb)
        
    # Replace '.' with ',' in file name since LaTeX doesn't like periods
    file_name = ff.replace_periods_with_commas(name)
    
    chdir('Figures/SRG_operator_evolution')
    f.savefig(file_name+'.pdf', bbox_inches='tight')
    chdir(cwd)

In [ ]:
def continuum_state_momentum_distribution(eps, kvnn, kmax, kmid, ntot, generator, lamb, lambda_bd=0.00):
    """
    Plots a semi-log figure of the momentum distribution |psi_eps(k)|^2 [fm^3] as a function of momentum k 
    [fm^-1] with psi_eps(0) from the bare Hamiltonian and psi_eps(lambda) from an evolved Hamiltonian at a given 
    value of lambda. Epsilon specifies the energy of the state (i.e. epsilon = -2.22 MeV would corrspond to 
    deuteron). Must specify details of the SRG-evolved potential, hence the arguments kvnn, channel, etc.
    
    Parameters
    ----------
    eps : float
        Energy value in units MeV.
    kvnn : int
        This number specifies the potential.
    kmax : float
        Maximum value in the momentum mesh.
    kmid : float
        Mid-point value in the momentum mesh.
    ntot : int
        Number of momentum points in mesh.
    generator : str
        SRG generator 'Wegner', 'T', or 'Block-diag'.
    lamb : float
        Evolution parameter lambda in units fm^-1.
    lambda_bd : float, optional
        Lambda value for block-diagonal decoupling (e.g. 2.00 fm^-1).

    Saves
    -----
    file_name = continuum_state_momentum_distribution_ with an extension based off the given arguments.
    file_type = .pdf
    
    """
    
    # Current working directory
    cwd = getcwd()
    
    
    # Load momentum distributions
    
    # Channel is 3S1
    channel = '3S1'
    
    # Load initial Hamiltonian, momentum, and weights
    H_initial = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
    k_array, k_weights = lp.load_momentum(kvnn, channel, kmax, kmid, ntot) 
    
    # Load unitary transformation
    # SRG calls function which builds U(s) out of un-evolved and evolved eigenvectors
    H_evolved = lp.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, 'srg', generator, lamb, lambda_bd)
    U_matrix = SRG_unitary_transformation(H_initial, H_evolved)
        
    # Compute initial and evolved wave functions
    psi_initial = ob.wave_function(H_initial, eps)
    u_initial = psi_initial[:ntot] # 3S1 component
    w_initial = psi_initial[ntot:] # 3D1 component
    
    psi_evolved = ob.wave_function(H_initial, eps, U=U_matrix)
    u_evolved = psi_evolved[:ntot] # 3S1 component
    w_evolved = psi_evolved[ntot:] # 3D1 component
    
    # Initial and evolved momentum distribution (divide by momenta and weights for mesh-independent result)
    phi_squared_initial = ( u_initial**2 + w_initial**2 ) / ( k_array**2 * k_weights )
    phi_squared_evolved = ( u_evolved**2 + w_evolved**2 ) / ( k_array**2 * k_weights )
    
    
    # Plot specifications

    # Limits of x and y axes
    xlim = [0.0, 4.0]
    ylim = [1e-6, 1e2]
    
    # Labels
    k_label = r'$k \/ [fm^{-1}]$'
    phi_squared_label = r'$\phi_{\epsilon}^2 \/ \/ [fm^3]$'
    lamb_label = r'$\lambda=%.1f \/ \/ fm^{-1}$'
    if generator == 'Block-diag':
        generator_label = r'$G=H_{BD}$'+'\n'+r'$\Lambda=%.2f \/ fm^{-1}$'%lambda_bd
    elif generator == 'Wegner':
        generator_label = r'$G=H_{D}$'
    elif generator == 'T':
        generator_label = r'$G=T_{rel}$'
    
    # Fontsize for labels
    legend_label_size = 16
    k_label_size = 18
    phi_squared_label_size = 20
    generator_label_size = 18
    
    # Location of labels
    legend_label_location = 'upper left'
    generator_label_location = 'upper right'
    
    
    # Plot the semi-log figure
    plt.close('all')
    
    f, ax = plt.subplots()
    
    ax.semilogy(k_array, phi_squared_evolved, color='xkcd:red', label=lamb_label%lamb)
    ax.semilogy(k_array, phi_squared_initial, color='xkcd:black', linestyle='dotted',
                label=r'$\lambda=\infty \/ \/ fm^{-1}$')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.legend(loc=legend_label_location, frameon=False, fontsize=legend_label_size)
    ax.set_xlabel(k_label, fontsize=k_label_size)
    ax.set_ylabel(phi_squared_label, fontsize=phi_squared_label_size)
    anchored_text_1 = AnchoredText(generator_label, prop=dict(size=generator_label_size),
                                   loc=generator_label_location, frameon=False)
    ax.add_artist(anchored_text_1)
    
    
    # Save figure
    
    # Name of the file
    if generator == 'Block-diag':
        name = 'continuum_state_momentum_distribution_eps%.1f_kvnn%d_%s%.2f_lamb%.1f'%(eps, kvnn, generator, 
                                                                                       lambda_bd, lamb)
    else:
        name = 'continuum_state_momentum_distribution_eps%.1f_kvnn%d_%s_lamb%.1f'%(eps, kvnn, generator, lamb)
        
    # Replace '.' with ',' in file name since LaTeX doesn't like periods
    file_name = ff.replace_periods_with_commas(name)
        
    chdir('Figures/SRG_operator_evolution')
    f.savefig(file_name+'.pdf', bbox_inches='tight')
    chdir(cwd)

__Potential evolution__

* Plotting functions for a potential SRG-evolving evolving in $\lambda$.

In [ ]:
def potential_contours(kvnn, channel, kmax, kmid, ntot, generator, lambda_bd=0.00):
    """
    Plots a 1 x 4 sub-plots figure of an NN potential evolving from lambda = 6.0 to 1.5 fm^-1 as contour plots. Must
    specify details of the SRG-evolved potential, hence the arguments kvnn, channel, etc.
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    channel : str
        The partial wave channel ('1S0', '3S1', etc.)
    kmax : float
        Maximum value in the momentum mesh.
    kmid : float
        Mid-point value in the momentum mesh.
    ntot : int
        Number of momentum points in mesh.
    generator : str
        SRG generator 'Wegner', 'T', or 'Block-diag'.
    lambda_bd : float, optional
        Lambda value for block-diagonal decoupling (e.g. 2.00 fm^-1).

    Saves
    -----
    file_name = potential_contours_ with an extension based off the given arguments.
    file_type = .pdf
    
    """
    
    # Current working directory
    cwd = getcwd()
    
    # Limits of axes on contours (units are fm^-1)
    axes_max = 4.0
    
    
    # Store evolved potentials
    
    # Load momentum for interpolation
    k_array, _ = lp.load_momentum(kvnn, channel, kmax, kmid, ntot)
    
    # Loop over lambda values in lambdas and save evolved potentials to dictionary where the keys are lambda values
    lambdas = [6.0, 3.0, 2.0, 1.5]
    
    # Initialize dictionary
    d = {}
    
    for lamb in lambdas:
        
        # Load evolved potential
        V_evolved = lp.load_potential(kvnn, channel, kmax, kmid, ntot, 'srg', generator, lamb, lambda_bd)
        
        # Interpolate the potential through 0 to axes_max for smoother looking figure (the extension _int means 
        # interpolated)
        k_array_int, V_evolved_int = ff.interpolate_matrix(k_array, V_evolved, axes_max)
        
        # Save potential to dictionary
        d[lamb] = V_evolved_int
    
    
    # Plot specifications
    
    # Labels
    k_label = r'$k \/ \/ [fm^{-1}]$'
    if generator == 'Block-diag':
        generator_label = r'$G=H_{BD}$'+'\n'+r'$\Lambda=%.2f \/ fm^{-1}$'%lambda_bd
    elif generator == 'Wegner':
        generator_label = r'$G=H_{D}$'
    elif generator == 'T':
        generator_label = r'$G=T_{rel}$'
    lamb_label = r'$\lambda=%.1f \/ fm^{-1}$'
    colorbar_label = r'$[fm]$'
    
    # Fontsize for labels and tick marks
    axes_label_size = 18
    generator_label_size = 17
    lambda_label_size = 17
    colorbar_label_size = 20
    axes_tick_size = 18
    colorbar_tick_size = 18

    # Location of labels
    generator_location = 'upper right'
    lamb_location = 'lower left'
        
    # Color scheme for contour plots
    color_style = 'jet'
    
    # Limits of color bar in fm
    mx = 1.0
    mn = -1.0
        
    # Step-size in labeling tick marks
    axes_stepsize = 1.0
    
    # x and y axes ticks
    axes_ticks = np.arange(0.0, axes_max + axes_stepsize, axes_stepsize)
    
    # Size of figure
    row_number = 1
    column_number = 4
    figure_size = (4*column_number, 3.5*row_number) # (width, height) - extra width for colorbar
    
    
    # Plot the 1 x 4 sub-plots figure     
    plt.close('all')
    
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharex=True, sharey=True, figsize=figure_size)
    
    # Sub-plot 1: lambda = 6.0 fm^-1
    ax1.pcolormesh(k_array_int, k_array_int, d[ lambdas[0] ], cmap=color_style, vmin=mn, vmax=mx, rasterized=True)
    # Set limits of axes
    ax1.set_xlim( (0, axes_max) )
    ax1.set_ylim( (0, axes_max) )
    # Set x and y labels
    ax1.set_xlabel(k_label, fontsize=axes_label_size)
    ax1.set_ylabel(k_label, fontsize=axes_label_size)
    # Specify axes tick marks
    ax1.xaxis.set_ticks(axes_ticks)
    ax1.xaxis.set_ticklabels(axes_ticks)
    ax1.yaxis.set_ticks(axes_ticks)
    ax1.yaxis.set_ticklabels(axes_ticks)
    # Position of x-axis label and tick marks
    ax1.xaxis.set_label_position('top')
    ax1.xaxis.tick_top()
    ax1.tick_params(labeltop=True, labelsize=axes_tick_size)
    # Prevent overlapping x-axis tick marks
    xticks_1 = ax1.xaxis.get_major_ticks()
    xticks_1[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_1 = AnchoredText(lamb_label%lambdas[0], prop=dict(size=lambda_label_size), loc=lamb_location)
    ax1.add_artist(anchored_text_1)
    
    # Sub-plot 2: lambda = 3.0 fm^-1
    ax2.pcolormesh(k_array_int, k_array_int, d[ lambdas[1] ], cmap=color_style, vmin=mn, vmax=mx, rasterized=True)
    # Set limits of axes
    ax2.set_xlim( (0, axes_max) )
    ax2.set_ylim( (0, axes_max) )
    # Set x label
    ax2.set_xlabel(k_label, fontsize=axes_label_size)
    # Specify axes tick marks
    ax2.xaxis.set_ticks(axes_ticks)
    ax2.xaxis.set_ticklabels(axes_ticks)
    # Position of x-axis label and tick marks
    ax2.xaxis.set_label_position('top')
    ax2.xaxis.tick_top()
    ax2.tick_params(labeltop=True, labelsize=axes_tick_size)
    # Prevent overlapping x-axis tick marks
    xticks_2 = ax2.xaxis.get_major_ticks()
    xticks_2[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_2 = AnchoredText(lamb_label%lambdas[1], prop=dict(size=lambda_label_size), loc=lamb_location)
    ax2.add_artist(anchored_text_2)
    
    # Sub-plot 3: lambda = 2.0 fm^-1
    ax3.pcolormesh(k_array_int, k_array_int, d[ lambdas[2] ], cmap=color_style, vmin=mn, vmax=mx, rasterized=True)
    # Set limits of axes
    ax3.set_xlim( (0, axes_max) )
    ax3.set_ylim( (0, axes_max) )
    # Set x label
    ax3.set_xlabel(k_label, fontsize=axes_label_size)
    # Specify axes tick marks
    ax3.xaxis.set_ticks(axes_ticks)
    ax3.xaxis.set_ticklabels(axes_ticks)
    # Position of x-axis label and tick marks
    ax3.xaxis.set_label_position('top')
    ax3.xaxis.tick_top()
    ax3.tick_params(labeltop=True, labelsize=axes_tick_size)
    # Prevent overlapping x-axis tick marks
    xticks_3 = ax3.xaxis.get_major_ticks()
    xticks_3[-1].set_visible(False)
    # Add generator and lambda labels onto plot as anchored texts
    anchored_text_3 = AnchoredText(lamb_label%lambdas[2], prop=dict(size=lambda_label_size), loc=lamb_location)
    ax3.add_artist(anchored_text_3)
    anchored_text_4 = AnchoredText(generator_label, prop=dict(size=generator_label_size), loc=generator_location)
    ax3.add_artist(anchored_text_4)

    # Sub-plot 4: lambda = 1.5 fm^-1
    c = ax4.pcolormesh(k_array_int, k_array_int, d[ lambdas[3] ], cmap=color_style, vmin=mn, vmax=mx, 
                       rasterized=True)
    # Set limits of axes
    ax4.set_xlim( (0, axes_max) )
    ax4.set_ylim( (0, axes_max) )
    # Set x label
    ax4.set_xlabel(k_label, fontsize=axes_label_size)
    # Specify axes tick marks
    ax4.xaxis.set_ticks(axes_ticks)
    ax4.xaxis.set_ticklabels(axes_ticks)
    # Position of x-axis label and tick marks
    ax4.xaxis.set_label_position('top')
    ax4.xaxis.tick_top()
    ax4.tick_params(labeltop=True, labelsize=axes_tick_size)
    # Add lambda label onto plot as anchored text
    anchored_text_5 = AnchoredText(lamb_label%lambdas[3], prop=dict(size=lambda_label_size), loc=lamb_location)
    ax4.add_artist(anchored_text_5)

    # Invert y-axis
    plt.gca().invert_yaxis()
    # Amount of white space in-between sub-plots
    f.subplots_adjust(hspace=0.0, wspace=0.0)
    # Adjust for colorbar space
    f.subplots_adjust(right=0.8)
    cbar_ax = f.add_axes( (0.85, 0.15, 0.05, 0.7) )
    # Add colorbar and set tick size
    cbar = f.colorbar(c, cax=cbar_ax)
    cbar.ax.tick_params(labelsize=colorbar_tick_size)
    # Set colorbar label
    cbar.ax.set_title(colorbar_label, fontsize=colorbar_label_size)

    
    # Save figure as a .pdf
    
    # Name of the file
    if generator == 'Block-diag':
        name = 'potential_contours_kvnn%d_%s_%s%.2f'%(kvnn, channel, generator, lambda_bd)
        # Replace '.' with ',' in file name since LaTeX doesn't like periods
        file_name = ff.replace_periods_with_commas(name)
    else:
        file_name = 'potential_contours_kvnn%d_%s_%s'%(kvnn, channel, generator)
        
    chdir('Figures/SRG_operator_evolution')
    f.savefig(file_name+'.pdf', bbox_inches='tight')
    chdir(cwd)

In [ ]:
def potential_diagonals_test(kvnn, channel, kmax_1, kmid_1, ntot_1, kmax_2, kmid_2, ntot_2, generator, lambda_bd=0.00):
    """
    Plots a 1 x 4 sub-plots figure comparing two NN potentials using different momentum meshes evolving from 
    lambda = 6.0 to 1.5 fm^-1 showing the diagonal matrix elements V(k, k). Must specify details of the SRG-evolved 
    potential, hence the arguments kvnn, channel, etc.
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    channel : str
        The partial wave channel ('1S0', '3S1', etc.)
    kmax_1 : float
        Maximum value in the first momentum mesh.
    kmid_1 : float
        Mid-point value in the first momentum mesh.
    ntot_1 : int
        Number of momentum points in first mesh.
    kmax_2 : float
        Maximum value in the second momentum mesh.
    kmid_2 : float
        Mid-point value in the second momentum mesh.
    ntot_2 : int
        Number of momentum points in second mesh.
    generator : str
        SRG generator 'Wegner', 'T', or 'Block-diag'.
    lambda_bd : float, optional
        Lambda value for block-diagonal decoupling (e.g. 2.00 fm^-1).
    
    """
    
    # Current working directory
    cwd = getcwd()
    
    
    # Store evolved potentials
    
    # Load momentum arrays
    k_array_1, _ = lp.load_momentum(kvnn, channel, kmax_1, kmid_1, ntot_1)
    k_array_2, _ = lp.load_momentum(kvnn, channel, kmax_2, kmid_2, ntot_2)
    
    # Loop over lambda values in lambdas and save evolved potentials to dictionary where the keys are lambda values
    lambdas = [6.0, 3.0, 2.0, 1.5]
    
    # Initialize dictionaries
    d = {}
    d[1] = {}
    d[2] = {}
    
    for lamb in lambdas:
        
        # Load evolved potential
        V_evolved_1 = lp.load_potential(kvnn, channel, kmax_1, kmid_1, ntot_1, 'srg', generator, lamb, lambda_bd)
        V_evolved_2 = lp.load_potential(kvnn, channel, kmax_2, kmid_2, ntot_2, 'srg', generator, lamb, lambda_bd)
        
        # Save diagonal elements of potentials to dictionary
        d[1][lamb] = np.diag(V_evolved_1[:ntot_1, :ntot_1])
        d[2][lamb] = np.diag(V_evolved_2[:ntot_2, :ntot_2])
        

    # Plot specifications
    
    # Labels
    x_label = r'$k \/ \/ [fm^{-1}]$'
    y_label = r'$V_{\lambda}(k,k) \/ \/ [fm]$'
    if generator == 'Wegner':
        generator_label = r'$G=H_{D}$'
    elif generator == 'T':
        generator_label = r'$G=T_{rel}$'
    # kvnn_label labels the EFT cutoff Lambda
    lamb_label = r'$\lambda=%.1f \/ fm^{-1}$'
    
    # Fontsize for labels and tick marks
    x_label_size = 20
    y_label_size = 24
    generator_label_size = 22
    lamb_label_size = 18
    legend_label_size = 16
    axes_tick_size = 14
     
    # Location of labels
    generator_label_location = 4
    legend_label_location = 4
    lamb_label_location = 9
    
    # Limits of axes
    x_max = 3.0 # Units of fm^-1
    if generator == 'Wegner':
        y_max = 2.5
        y_min = -4.0
    elif generator == 'T':
        y_max = 1.5
        y_min = -5.0
        
    # Step-size in labeling tick marks
    x_stepsize = 0.5
    y_stepsize = 1
    
    # x and y axes ticks
    x_ticks = np.arange(0.0, x_max + x_stepsize, x_stepsize)
    y_ticks = np.arange(y_min, y_max - 0.5 + y_stepsize, y_stepsize)
    
    # Size of figure
    row_number = 1
    column_number = 4
    figure_size = (3.5*column_number, 3.5*row_number) # (width, height)
    
    
    # Plot the 1 x 4 sub-plots figure     
    plt.close('all')
    
    f, (ax1, ax2, ax3, ax4) = plt.subplots(row_number, column_number, sharex=True, sharey=True, figsize=figure_size)
    
    # Sub-plot 1: lambda = 6.0 fm^-1
    ax1.plot(k_array_1, d[1][ lambdas[0] ], color='xkcd:red', label='Mesh 1')
    ax1.plot(k_array_2, d[2][ lambdas[0] ], color='xkcd:black', linestyle='dotted', label='Mesh 1')
    # Set limits of axes
    ax1.set_xlim( (0, x_max) )
    ax1.set_ylim( (y_min, y_max) )
    # Set y label
    ax1.set_ylabel(y_label, fontsize=y_label_size)
    # Specify y axis tick marks
    ax1.yaxis.set_ticks(y_ticks)
    ax1.yaxis.set_ticklabels(y_ticks)
    # Set fontsize of tick marks
    ax1.tick_params(labelsize=axes_tick_size)
    # Prevent overlapping axes tick marks
    xticks_1 = ax1.xaxis.get_major_ticks()
    xticks_1[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_1 = AnchoredText(lamb_label%lambdas[0], prop=dict(size=lamb_label_size), loc=lamb_label_location,
                                   frameon=False)
    ax1.add_artist(anchored_text_1)
    # Set legend
    ax1.legend(loc=legend_label_location, frameon=False, fontsize=legend_label_size)
    
    # Sub-plot 2: lambda = 3.0 fm^-1
    ax2.plot(k_array_1, d[1][ lambdas[1] ], color='xkcd:red')
    ax2.plot(k_array_2, d[2][ lambdas[1] ], color='xkcd:black', linestyle='dotted')
    # Set limits of axes
    ax2.set_xlim( (0, x_max) )
    ax2.set_ylim( (y_min, y_max) )
    # Set fontsize of tick marks
    ax2.tick_params(labelsize=axes_tick_size)
    # Prevent overlapping axes tick marks
    xticks_2 = ax2.xaxis.get_major_ticks()
    xticks_2[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_2 = AnchoredText(lamb_label%lambdas[1], prop=dict(size=lamb_label_size), loc=lamb_label_location,
                                   frameon=False)
    ax2.add_artist(anchored_text_2)
    
    # Sub-plot 3: lambda = 2.0 fm^-1
    ax3.plot(k_array_1, d[1][ lambdas[2] ], color='xkcd:red')
    ax3.plot(k_array_2, d[2][ lambdas[2] ], color='xkcd:black', linestyle='dotted')
    # Set limits of axes
    ax3.set_xlim( (0, x_max) )
    ax3.set_ylim( (y_min, y_max) )
    # Set fontsize of tick marks
    ax3.tick_params(labelsize=axes_tick_size)
    # Prevent overlapping axes tick marks
    xticks_3 = ax3.xaxis.get_major_ticks()
    xticks_3[-1].set_visible(False)
    # Add lambda label onto plot as anchored text
    anchored_text_3 = AnchoredText(lamb_label%lambdas[2], prop=dict(size=lamb_label_size), loc=lamb_label_location,
                                   frameon=False)
    ax3.add_artist(anchored_text_3)
    
    # Sub-plot 4: lambda = 1.5 fm^-1
    ax4.plot(k_array_1, d[1][ lambdas[3] ], color='xkcd:red')
    ax4.plot(k_array_2, d[2][ lambdas[3] ], color='xkcd:black', linestyle='dotted')
    # Set limits of axes
    ax4.set_xlim( (0, x_max) )
    ax4.set_ylim( (y_min, y_max) )
    # Set fontsize of tick marks
    ax4.tick_params(labelsize=axes_tick_size)
    # Add lambda label onto plot as anchored text
    anchored_text_4 = AnchoredText(lamb_label%lambdas[3], prop=dict(size=lamb_label_size), loc=lamb_label_location,
                                   frameon=False)
    ax4.add_artist(anchored_text_4)
    anchored_text_5 = AnchoredText(generator_label, prop=dict(size=generator_label_size),
                                   loc=generator_label_location, frameon=False)
    ax4.add_artist(anchored_text_5)

    # Amount of white space in-between sub-plots
    f.subplots_adjust(hspace=0.0, wspace=0.0)

    
    # Testing so no need to save
    plt.show()